<a href="https://colab.research.google.com/github/serdarcekinmez/Fine_tuner_40_books/blob/master/Copie_de_clean_finetune_and_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 88.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 22.3 MB/s eta 0:00:00


In [ ]:
!git config --global user.email "serdarcekinmez@gmail.com"
!git config --global user.name "serdarcekinmez"

from huggingface_hub import notebook_login

notebook_login()

!apt install git-lfs








Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:

import transformers

from datasets import load_dataset

datasets = load_dataset("text", data_files={"train": "/content/TrainCorpus.txt", "validation": "/content/ValidationCorpus.txt"})




model_checkpoint = "facebook/opt-350m"

from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

def tokenize_function(examples):
    return tokenizer(examples["text"])

tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

# block_size = tokenizer.model_max_length
block_size = 128

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result


lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

#trainer model

from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

from transformers import Trainer, TrainingArguments

model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-sixbooks",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

trainer.train()


# Evaluating:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

trainer.push_to_hub()
finetuned_model = model
finetuned_model.save_pretrained(save_directory, push_to_hub=True, repo_name="CSerdar014191/opt-350m-sixhorrorbooks")

# Push the tokenizer to the hub
tokenizer.push_to_hub(repo_id="CSerdar014191/opt-350m-sixhorrorbooks")


#no need but as example against above two:  finetuned_model.save_pretrained(save_directory, push_to_hub=True, repo_name="CSerdar014191/opt-350m-sixhorrorbooks")






Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-5904f231a8715bec/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

...

Upload file pytorch_model.bin:   0%|          | 1.00/1.23G [00:00<?, ?B/s]

Upload file runs/Apr16_11-25-28_6919415afeb1/events.out.tfevents.1681644945.6919415afeb1.4999.2:   0%|        …

Upload file runs/Apr16_11-25-28_6919415afeb1/events.out.tfevents.1681644632.6919415afeb1.4999.0:   0%|        …

To https://huggingface.co/CSerdar014191/opt-350m-sixbooks
   fb9c604..160bcd6  main -> main

   fb9c604..160bcd6  main -> main

To https://huggingface.co/CSerdar014191/opt-350m-sixbooks
   160bcd6..f9e4a28  main -> main

   160bcd6..f9e4a28  main -> main



TypeError: ignored

In [ ]:

# Evaluating:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

trainer.push_to_hub()
save_directory = '/content/opt-350m-sixbooks'
finetuned_model = model

# Push the tokenizer to the hub
finetuned_model.save_pretrained(save_directory, push_to_hub=True, repo_id="CSerdar014191/opt-350m-sixhorrorbooks")

tokenizer.push_to_hub(repo_id="CSerdar014191/opt-350m-sixhorrorbooks")




Perplexity: 25.43


Upload file runs/Apr16_11-25-28_6919415afeb1/events.out.tfevents.1681644945.6919415afeb1.4999.2: 100%|########…

To https://huggingface.co/CSerdar014191/opt-350m-sixbooks
   f4701ec..a6be40c  main -> main

   f4701ec..a6be40c  main -> main



Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/1.32G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/CSerdar014191/opt-350m-sixhorrorbooks/commit/6d63810a0a5aa98da98a119eaa4cbdb8579bdda4', commit_message='Upload tokenizer', commit_description='', oid='6d63810a0a5aa98da98a119eaa4cbdb8579bdda4', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from transformers import GPT2TokenizerFast

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
tokenizer.push_to_hub("CSerdar014191/opt-350m-sixbooks")


CommitInfo(commit_url='https://huggingface.co/CSerdar014191/opt-350m-sixbooks/commit/d97cbe3b018d2987239e683aa5327bb459d68851', commit_message='Upload tokenizer', commit_description='', oid='d97cbe3b018d2987239e683aa5327bb459d68851', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:

# Generate model
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("CSerdar014191/opt-350m_test08_tuner", from_tf=True)
tokenizer = AutoTokenizer.from_pretrained("CSerdar014191/opt-350m_test08_tuner")

prompt = 'He was horrified'
inputs = tokenizer(prompt, return_tensors="pt")
#outputs = model.generate(**inputs, do_sample=True, min_new_tokens=80, max_new_tokens=100, temperature=1, top_k=50, top_p=0.95, num_return_sequences=1)

#for output in outputs:
    #print(tokenizer.decode(output, skip_special_tokens=True))

# ...

outputs = model.generate(**inputs, do_sample=True, min_new_tokens=40, max_new_tokens=80, temperature=1.8, top_k=50, top_p=0.95, num_return_sequences=2)
decoded_outputs = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

for output in outputs:
    print(tokenizer.decode(output, skip_special_tokens=True))



### Note for the second model, OPTForCausalLM is not included in transformers.
### Try by replacing above GPT2LMHeadModel; if it does not work, then search for a way to run it.







All TF 2.0 model weights were used when initializing OPTForCausalLM.

All the weights of OPTForCausalLM were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use OPTForCausalLM for predictions without further training.


He was horrified to believe to
reassessment that there seemed any use on her--I think
Follow in my way as always, and we cannot

_IV September_:
CHAPTER XVII

He was horrified, seeing by the sight of his friend a hideous and malcomorous man-


 By the meantime his friends had
brought two people who stood between the fire and coming towards me--Hendrix and



In [ ]:
# model2
from transformers import GPT2LMHeadModel

model2 = GPT2LMHeadModel.from_pretrained("CSerdar014191/opt-350m_test08_tuner")
tokenizer2 = AutoTokenizer.from_pretrained("CSerdar014191/opt-350m_test08_tuner")

prompt = 'I was on a ship'
inputs = tokenizer(prompt, return_tensors="pt")
outputs2 = model2.generate(**inputs, do_sample=True, min_new_tokens=50, max_new_tokens=70, temperature=0.7, top_k=50, top_p=0.95, num_return_sequences=3)
decoded_outputs2 = [tokenizer.decode(output) for output in outputs2]
print(decoded_outputs2)


You are using a model of type opt to instantiate a model of type gpt2. This is not supported for all configurations of models and can yield errors.


OSError: ignored

In [ ]:
# this part of code is just to visualize how dataset looks like
from datasets import ClassLabel 
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))